In [177]:
# 사용 모듈 import
import pymysql
import csv
import pandas as pd
import config

import folium

In [180]:
# 데이터베이스 연동

db = pymysql.connect(
    host=config.DATABASE_CONFIG['host'], 
	port=3306, 
	user=config.DATABASE_CONFIG['user'],
	passwd=config.DATABASE_CONFIG['password'],
	db=config.DATABASE_CONFIG['db'],charset='utf8',autocommit=True
)

cursor = db.cursor()

In [67]:
# 테이블 생성(시설 테이블)

cursor.execute("""CREATE TABLE mapdb.facilities (
  num int AUTO_INCREMENT NOT NULL,
  name VARCHAR(45) NOT NULL,
  phone varchar(15) NULL,
  address VARCHAR(110) NOT NULL,
  longitude FLOAT(13, 10) NULL,
  latitude FLOAT(12, 10) NULL,
  big_address VARCHAR(45) NOT NULL,
  PRIMARY KEY (num));""")



In [111]:
# 데이터 셋 가공(시설 정보), 시설정보 테이블에 저장
# name, address 경도, 위도, 넓은 범위 주소

facilities = []

sql = "INSERT INTO facilities (name, phone, address, latitude, longitude, big_address) VALUES (%s, %s, %s, %s, %s, %s)"

# csv파일로부터 데이터셋 데이터베이스에 저장
with open('facilities.csv', 'r') as fileRead:
    reader = csv.reader(fileRead)
    i = 0
    for line in reader:
        facilities.append(line)
        facilities[i][2] = facilities[i][2][4:]
        temparr = facilities[i][2].split(" ")
        tempadd = temparr[0] + " " + temparr[1]
        facilities[i].append(tempadd)
        if facilities[i][3] == "":
            facilities[i][3] = facilities[i][4] = 0
        if i != 0 :
            cursor.execute(sql, (facilities[i][0], facilities[i][1], facilities[i][2], facilities[i][3], facilities[i][4], facilities[i][5]))
        i += 1
    

cursor.close()

In [86]:
# 테이블 생성(지역 테이블)

cursor.execute("""
CREATE TABLE mapdb.sectors (
    big_address VARCHAR(45) NOT NULL PRIMARY KEY,
    longitude FLOAT(13, 10) NULL,
    latitude FLOAT(12, 10) NULL);
""")

3


In [131]:
# 테이블 생성(예방접종 예약)

cursor.execute("""
CREATE TABLE mapdb.reservations(
    name VARCHAR(10) NOT NULL,
    date DATE NOT NULL,
    hospital VARCHAR(45) NOT NULL,
    phone VARCHAR(15) NULL,
    address VARCHAR(110) NOT NULL,
    fasnum INT NOT NULL,
    PRIMARY KEY (name, fasnum),
    CONSTRAINT reserve
     FOREIGN KEY (fasnum)
     REFERENCES mapdb.facilities(num) 
     ON UPDATE NO ACTION
     ON DELETE NO ACTION);
""")

0

In [119]:
# 테스트

cursor.execute("describe sectors;")
suc = cursor.fetchall()
print(suc)

(('big_address', 'varchar(45)', 'NO', 'PRI', None, ''), ('longitude', 'float(13,10)', 'YES', '', None, ''), ('latitude', 'float(12,10)', 'YES', '', None, ''))


In [92]:
# 데이터 셋 가공(지역 정보)

cursor.execute("""
SELECT big_address, AVG(longitude) AS longitude, AVG(latitude) as latitude 
FROM facilities
WHERE longitude != 0 AND latitude != 0
GROUP BY big_address;""")

sector = cursor.fetchall()

print(sector[0])

('수원시 장안구', 127.00037107362851, 37.29908844664857)


In [94]:
# 지역정보 테이블에 저장

sql = "INSERT INTO sectors (big_address,longitude, latitude) VALUES (%s, %s, %s)"

for sec in sector:
    cursor.execute(sql, (sec[0], sec[1], sec[2]))
    

In [ ]:
# 테이블 참조를 위해 데이터 삭제, 데이터베이스 수정 후 재입력

# 삭제
cursor.execute("truncate facilities;")

# 외래키 설정
cursor.execute("""
ALTER TABLE facilities
ADD CONSTRAINT search
FOREIGN KEY (big_address)
REFERENCES sectors(big_address);
""")

# 앞서 데이터 입력 재실행

In [ ]:
# 시설 릴레이션의 외래키 설정

cursor.execute("""ALTER TABLE facilities
                  ADD CONSTRAINT search
                  FOREIGN KEY (big_address)
                  REFERENCES sectors (big_address);""")

In [118]:
# 검색의 편의성을 위해 시설 릴레이션의 외래키에 인덱스 추가

cursor.execute("ALTER TABLE facilities ADD INDEX addressSearch(big_address);")


0

In [165]:
# 지도 탐색 함수 설계

data = []

# 지역검색 함수
def sectorSearch():
    sql = "SELECT * FROM facilities WHERE big_address=%s;"
    sec = input("지역 입력 : ")
    cursor.execute(sql, sec)
    result = cursor.fetchall() # 데이터베이스로 부터 불러온 데이터 저장
    createDataframe(result)
    if len(data) == 0:
        print("지역 정보가 없습니다.")
        return
    df = pd.DataFrame(data)
    df = df.astype({'위도' : np.float,'경도' : np.float})
    return (printMap(df, sec), df)
    
    
# 검색한 지역에 대한 시설 데이터를 가공하는 함수
# 0번호, 1시설 명, 2전화번호, 3주소, 4경도, 5위도, 6지역
def createDataframe(result):
    global data
    for facil in result:
        temp = {"시설 번호": facil[0], "시설 명": facil[1], "전화번호": facil[2], "도로명 주소": facil[3], "경도": facil[4], "위도": facil[5]}
        data.append(temp)
    
    
# 입력값을 바탕으로 지도를 출력하는 함수
def printMap(df, sec):
    sql = "SELECT longitude, latitude FROM sectors WHERE big_address=%s"
    cursor.execute(sql, sec)
    result = cursor.fetchone()
    map = folium.Map(location=[result[1] ,result[0]],zoom_start=14)
    for i in range(0, df.shape[0]) :
        folium.Circle([df['위도'][i] ,df['경도'][i]],popup = df['시설 명'][i],
                       radius = 15, color = "#00f",fill_color = '#fff').add_to(map)
    map.save('result.html')
    return map
    
    
(map, df) = sectorSearch()
map

지역 입력 : 수원시 영통구


In [160]:
df

,시설 번호,시설 명,전화번호,도로명 주소,경도,위도
0,20,365내과의원,031-233-2878,"수원시 영통구 봉영로 1569, (영통동, 뉴월드프라자 402,403호)",127.071022,37.252064
1,37,365아이랑소아청소년과의원,031-273-0203,"수원시 영통구 중부대로 604, (영통동, 수원영통리슈빌스카이오피스텔) 1층",127.081390,37.267296
2,55,365힐링의원,031-203-3650,"수원시 영통구 영통로 151, (망포동) 2층",127.055267,37.240944
3,162,고려웰가정의학과의원,031-217-8871,"수원시 영통구 대학로 109, (이의동, 골드클래스5) 302호",127.047165,37.305096
4,216,광교365메디컬의원,031-213-3655,"수원시 영통구 광교중앙로 170, (하동) 광교 효성해링턴 타워 313호,314호",127.060699,37.288124
...,...,...,...,...,...,...
94,3936,허브이비인후과의원,031-546-2848,"수원시 영통구 매영로310번길 87, (영통동) 동보.신명아파트 2층 208호 2...",127.064087,37.250664
95,3951,현대가정의학과의원,031-202-7969,"수원시 영통구 봉영로1770번길 21, (영통동, 신명.한국아파트) 216호",127.083084,37.267418
96,4028,홍진희소아청소년과의원,031-273-7582,"수원시 영통구 영통로214번길 9, (영통동)",127.057640,37.246578
97,4051,황인섭내과의원,031-214-8300,"수원시 영통구 매영로 85, (매탄동, 성일코아빌딩) 401호",127.052094,37.268158


In [174]:
# 예방접종 예약 함수 설계

def reservation():
    name = input("접종자 명을 입력 : ")
    facilNum = input("예방접종할 시설의 시설 번호를 입력 : ")
    date = input("접종 날짜를 입력(YY-MM-DD) : ")
    sql = "SELECT * FROM facilities WHERE num=%s"
    cursor.execute(sql, facilNum)
    facilInfo = cursor.fetchone()
    if len(facilInfo) == 0:
        print("없는 시설 번호 입니다.")
        return
    sql2 = "INSERT INTO reservations (name, date, hospital, phone, address, fasnum) VALUES (%s, %s, %s, %s, %s, %s)"    
    cursor.execute(sql2, (name, date, facilInfo[1], facilInfo[2], facilInfo[3], facilInfo[0]))
    
reservation()

접종자 명을 입력 : 김창민
예방접종할 시설의 시설 번호를 입력 : 20
접종 날짜를 입력(YY-MM-DD) : 21-12-10


In [176]:
cursor.close()